_The following cell contains secret tokens:_

In [1]:
# The code was removed by Watson Studio for sharing.

# Find a location to set up a local store that helps saving the planet!

## Introduction
### An environmental/cultural Problem
Fast fashion is the term used to describe clothing designs that move quickly from the catwalk to stores to meet new trends. The rise of fast fashion brands like Zara, Uniqlo, H&M, GAP and Topshop is accompanied with the huge increase of clothing production. According to a study of McKinsey & Company the worldwide clothing production doubled from 2000 to 2014 due to falling production costs, streamlined operations, and rising consumer spending.<sup>[1](https://www.mckinsey.com/business-functions/sustainability/our-insights/style-thats-sustainable-a-new-fast-fashion-formula)</sup> In Germany alone, 1,350,000 tons of textile waste arise yearly.<sup>[2](https://www.bvse.de/themen/geschichte-des-textilrecycling/zahlen-zur-sammlung-und-verwendung-von-altkleidern.html)</sup> 

Today 57% of German customers name sustainability as an importent buying criteria for clothing.<sup>[3](http://de.statista.com/statistik/daten/studie/955983/umfrage/umfrage-zu-wichtigen-kriterien-beim-kauf-von-bekleidung-in-deutschland)</sup> A break of the economic fast fashion trend towards sustainable and slow fashion is likely to happen in the next years.<sup>[4](https://www.fashionrevolution.org/),[5](https://www.greenpeace.de/sites/www.greenpeace.de/files/publications/s01951_greenpeace_report_konsumkollaps_fast_fashion.pdf),[6](https://fashion-week-berlin.com/en/blog/single-news/going-green-movement-on-german-catwalks.html)</sup> The reuse of already produced clothings is an approach of the Zero Waste and the Sustainable Fashion Movement to overcome the eco-unfriedly consumer behavior of the past years. Through the reuse of clothes and textiles, the pollution- and energy-intensive production of new clothing can be avoided.

With a combined 36% of the global share, the United States, the United Kingdom and Germany were the top three used clothes exporters in 2015. Because several countries in Eastern Africa and China announced to ban imports of textile waste, the purchasing prices on used clothings will most likely dramatically drop due to the collapse of the export market.<sup>[7](http://www.chinadaily.com.cn/china/2017-07/21/content_30194081.htm),[8](https://www.un.org/africarenewal/magazine/december-2017-march-2018/protectionist-ban-imported-used-clothing)</sup> 

### The Business Idea
The drop of purchasing prices for used clothings and the increase of consumer interest due to the zero waste/sustainability trend in German society, combined with a trend for vintage fashion leads to a relatively high profit margin. 

Therefore it's a great time to open up a second-hand store! Since our possible custumers care about the environment and their own carbon footprint, let's decide to set up a local store instead of an online shop. Berlin has a lot of tourists and multiple fashion fairs every year, e.g. Bread&Butter, Mecedes Benz-Fashion Week. It's a fashion hotspot in Germany and the Berlin Street Style is famous among fashionistas around the globe.<sup>[8](https://www.vogue.com/slideshow/berlin-fashion-week-fall-2018)</sup> 

### The Business Problem
But what's the best adress to set up our new store? In this project we will try to find an optimal location for a second-hand store. Specifically, this report will be targeted to stakeholders interested in opening a second-hand store in Berlin, Germany.

To benefit from the spending capacity of tourists visiting the city we want to determine a neighborhood that is well-frequented by them. Since another second-hand store nearby tends to draw the same demographic of customers and all of our products are unique in their style, size and story they tell - we can assume, that we can benefit from another second-hand store nearby through an increase of our customer volume/walk-in traffic. 

Berlin is known for it's high rents, thus the estimated rent per square metre will be a comparision criterion and a relevant information for the stakeholders. We will use Data Science methods to generate a list of the most promissing locations based on the mentioned criteria. Then we will express the (dis-)advantages of each location, so that the stakeholders are able to make an informed decision on where to set up the store.

## Data

For our analysis we need several data resources:

+ __Foursquare API:__ 
  + Find places that are popular with tourists
  + Find existing second-hand shops
  + Get address of potential new store location (reverse geocoding)
+ __GeoJSON Data:__ 
  + Borders of the boroughs and neighborhoods of Berlin
+ __[ImmobilienScout24 API](https://api.immobilienscout24.de/our-apis/search/realestate-specific-parameters.html#store):__ 
  + Get data on avg. rent price per square metre for region of location
  + (Get nearest store to rent)

In [2]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize 

In [3]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.



In [4]:
def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

In [5]:
### Get data on existing second-hand stores in Belrin, Germany ###  
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
    client_id=FOURSQUARE_CLIENT_ID,
    client_secret=FOURSQUARE_CLIENT_SECRET,
    v='20190610',
#  ll='40.7243,-74.0018',
    near='Berlin, Germany',
    radius=100000,
    categoryId='4bf58dd8d48988d101951735,52f2ab2ebcbc57f1066b8b30' # Second-Hand & Vintage Stores/Second-Hand Bookstores
#  query='clothing',
#  limit=1
)

results = requests.get(url=url, params=params).json()['response']['groups'][0]['items']

In [6]:
display(results)

[{'reasons': {'count': 0,
   'items': [{'reasonName': 'globalInteractionReason',
     'summary': 'This spot is popular',
     'type': 'general'}]},
  'referralId': 'e-0-4c7bf19b2d3ba1430e8195d0-0',
  'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/bookstore_',
      'suffix': '.png'},
     'id': '4bf58dd8d48988d114951735',
     'name': 'Bookstore',
     'pluralName': 'Bookstores',
     'primary': True,
     'shortName': 'Bookstore'}],
   'id': '4c7bf19b2d3ba1430e8195d0',
   'location': {'address': 'Wörther Str. 27',
    'cc': 'DE',
    'city': 'Berlin',
    'country': 'Deutschland',
    'crossStreet': 'Rykestr.',
    'formattedAddress': ['Wörther Str. 27 (Rykestr.)',
     '10405 Berlin',
     'Deutschland'],
    'labeledLatLngs': [{'label': 'display',
      'lat': 52.53597689739696,
      'lng': 13.421060493864418}],
    'lat': 52.53597689739696,
    'lng': 13.421060493864418,
    'postalCode': '10405',
    'state': 'Berlin'},
   'name': "Sain

In [7]:
df_venues = pd.DataFrame([[
            item['venue']['id'],
            item['venue']['name'],
            get_categories(item['venue']['categories'])[0][0],
            item['venue']['location']['lat'], 
            item['venue']['location']['lng'],
            #item['venue']['location']['postalCode'],
            format_address(item['venue']['location'])
          ] for item in results], columns=['id_foursquare', 'venue', 'venue_category', 'latitude', 'logitude', 'address'])

In [8]:
display(df_venues)

,id_foursquare,venue,venue_category,latitude,logitude,address
0,4c7bf19b2d3ba1430e8195d0,Saint George's,Bookstore,52.535977,13.421060,"Wörther Str. 27 (Rykestr.), 10405 Berlin"
1,4ee388e9f9abfc7225bfa20f,Let Them Eat Cake,Thrift / Vintage Store,52.483113,13.441568,"Weserstr. 164, 12045 Berlin"
2,509f919b011cc38cb9b3ce02,Vintage Galore,Thrift / Vintage Store,52.492404,13.426570,"Sanderstr. 12, 12047 Berlin"
3,4adcda8ff964a520284b21e3,Schönhauser Design,Thrift / Vintage Store,52.525408,13.406730,"Alte Schönhauser Str. 28, 10119 Berlin"
4,57a620a0cd10f20013ccbab3,V Vintage Fashion,Thrift / Vintage Store,52.510012,13.455918,"Kopernikusstrasse 18, 10245 Berlin"
5,4adcda8ff964a520024b21e3,Kleidermarkt Garage,Thrift / Vintage Store,52.501703,13.351815,"Ahornstr. 2, 10787 Berlin"
6,540afe74498e77fa823a120d,Repeater,Thrift / Vintage Store,52.489050,13.433036,"Pannierstraße 45 (Framstraße), Berlin"
7,4c5db3716147be9aa8d09209,Sing Blackbird,Thrift / Vintage Store,52.492545,13.426195,"Sanderstr. 11, 12047 Berlin"
8,4adcda79f964a520cc4621e3,Café Tasso,Café,52.515704,13.456953,"Frankfurter Allee 11, 10247 Berlin"
9,500b0183e4b00d76e4cd16f9,Pink Cadillac,Thrift / Vintage Store,52.498340,13.442615,"Wrangelstr. 55, 10997 Berlin"


In [9]:
### Generate a map of Berlin and mark the second hand stores ###
berlin_map = folium.Map(location=[52.524, 13.410], zoom_start=11)

for lat, lng, label in zip(df_venues.latitude, df_venues.logitude, df_venues.venue):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5, # define how big you want the circle markers to be
        color='#3186cc',
        fill=True,
        fill_color='#3186cc',
        popup=label,
        fill_opacity=0.6
    ).add_to(berlin_map)

berlin_map

In [10]:
berlin_zip_topo = requests.get('https://raw.githubusercontent.com/funkeinteraktiv/Berlin-Geodaten/master/berlin_postleitzahlen.topojson').json()

In [11]:
berlin_geo = requests.get('https://raw.githubusercontent.com/m-hoerz/berlin-shapes/master/berliner-bezirke.geojson').json()

In [12]:
folium.TopoJson(
    berlin_zip_topo,
    'objects.berlin_postleitzahlen',
    name='Postal Codes'
).add_to(berlin_map)

folium.GeoJson(
    berlin_geo,
    name='Boroughs'
).add_to(berlin_map)

folium.LayerControl().add_to(berlin_map)

berlin_map

In [13]:
berlin_map.choropleth(
    geo_data=berlin_zip_topo,
    topojson='objects.berlin_postleitzahlen',
    data=df_venues,
    columns=['venue']['zip'],
    key_on='feature.properties.spatial_alias',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name=''
)

# display map
berlin_map

TypeError: list indices must be integers or slices, not str

In [ ]:
def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues